In [2]:
%matplotlib inline
# Esto es para que las gráficas se muestren aquí mismo, en el notebook

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para poder hacer gráficos en 3D
import ipywidgets as widgets  # Esto es para hacer los controles interactivos
from IPython.display import display, clear_output  # Para mostrar cosas y limpiar la pantalla

mu0 = 4 * np.pi * 1e-7  # Constante magnética en el vacío, muy importante para calcular el campo

# Aquí definimos las cajas donde vamos a poner los valores que queremos usar
R_input = widgets.FloatText(value=1.0, description='Radio R (m):')  # Radio de la espira en metros
I_input = widgets.FloatText(value=1.0, description='Corriente I (A):')  # Corriente que pasa por la espira
q_input = widgets.FloatText(value=1e-6, description='Carga q (C):')  # La carga puntual que va a sentir la fuerza
x_input = widgets.FloatText(value=0.0, description='x carga:')  # Posición en x de la carga
y_input = widgets.FloatText(value=0.0, description='y carga:')  # Posición en y de la carga
z_input = widgets.FloatText(value=0.5, description='z carga:')  # Posición en z de la carga
vx_input = widgets.FloatText(value=0.0, description='vx:')  # Componente x de la velocidad de la carga
vy_input = widgets.FloatText(value=0.0, description='vy:')  # Componente y de la velocidad
vz_input = widgets.FloatText(value=0.0, description='vz:')  # Componente z de la velocidad

# Botón para decir que ya pusiste los valores y quieres seguir
ready_button = widgets.Button(description="Listo ✅", button_style='success')

# Botones para confirmar si quieres ver la simulación o no (toggle entre Sí y No)
confirm_button = widgets.ToggleButtons(options=["No", "Sí"], description='¿Seguro?', button_style='info')

# Área donde vamos a imprimir mensajes y mostrar la gráfica
output = widgets.Output()

# Esta función se ejecuta cuando presionas el botón "Listo"
def on_ready_clicked(b):
    with output:
        clear_output(wait=True)  # Limpiamos la pantalla antes de mostrar mensajes nuevos
        errores = []
        # Checamos que el radio no sea cero o negativo porque no tendría sentido
        if R_input.value <= 0:
            errores.append("❌ El radio R debe ser mayor que 0.")
        # Checamos que la carga tampoco sea cero, porque entonces no habría fuerza
        if q_input.value == 0:
            errores.append("❌ La carga q no puede ser cero.")
        # Si hubo errores los mostramos
        if errores:
            for e in errores:
                print(e)
        else:
            # Si todo está bien, preguntamos si quieres continuar con la simulación
            print("✅ Parámetros correctos. ¿Deseas continuar?")
            display(confirm_button)  # Mostramos el botón para confirmar

# Esta función es la que hace toda la simulación y gráfica
def simular():
    with output:
        clear_output(wait=True)  # Limpiamos todo para no mezclar con cosas anteriores
        print("🔄 Ejecutando simulación...")

        # Guardamos los valores que el usuario escribió para usarlos
        R = R_input.value
        I = I_input.value
        q = q_input.value
        r_carga = np.array([x_input.value, y_input.value, z_input.value])  # Posición de la carga
        v_carga = np.array([vx_input.value, vy_input.value, vz_input.value])  # Velocidad de la carga

        # Creamos los puntos de la espira, que es un círculo en el plano XY
        N = 100  # Cuántos pedacitos para aproximar la espira
        theta = np.linspace(0, 2*np.pi, N)  # Angulos para dibujar el círculo completo
        x = R * np.cos(theta)  # Coordenadas x de la espira
        y = R * np.sin(theta)  # Coordenadas y de la espira
        z = np.zeros(N)        # z=0 porque está en el plano XY
        espira = np.vstack((x, y, z)).T  # Matriz con las coordenadas de cada punto

        # Esta función calcula el campo magnético en cualquier punto usando Biot-Savart
        def campo_magnetico_biot_savart(espira, r_punto):
            B = np.zeros(3)  # Campo inicial es cero vectorialmente
            for i in range(len(espira)):
                r1 = espira[i]  # Un punto de la espira
                r2 = espira[(i + 1) % len(espira)]  # El siguiente punto (con ciclo para cerrar el círculo)
                dl = r2 - r1  # Vector diferencial de corriente
                r = r_punto - r1  # Vector desde el elemento dl hasta el punto donde queremos el campo
                r_norm = np.linalg.norm(r)  # Magnitud de ese vector
                if r_norm == 0:
                    continue  # Si el punto coincide, ignoramos para evitar división entre cero
                # Aplicamos la fórmula diferencial de Biot-Savart para campo magnético
                dB = mu0 * I / (4 * np.pi) * np.cross(dl, r) / (r_norm**3)
                B += dB  # Sumamos al campo total
            return B

        # Creamos una rejilla de puntos donde calcularemos el campo para visualizarlo
        X, Y, Z = np.meshgrid(
            np.linspace(-1.5, 1.5, 6),
            np.linspace(-1.5, 1.5, 6),
            np.linspace(-1.0, 1.0, 4)
        )
        BX, BY, BZ = np.zeros_like(X), np.zeros_like(Y), np.zeros_like(Z)  # Campos en cada punto

        # Calculamos el campo en cada punto de la rejilla
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                for k in range(X.shape[2]):
                    r_point = np.array([X[i, j, k], Y[i, j, k], Z[i, j, k]])
                    B_vec = campo_magnetico_biot_savart(espira, r_point)
                    BX[i, j, k], BY[i, j, k], BZ[i, j, k] = B_vec

        # Ahora calculamos el campo y la fuerza sobre la carga puntual
        B_carga = campo_magnetico_biot_savart(espira, r_carga)
        F_magn = q * np.cross(v_carga, B_carga)  # Fórmula F = q v × B

        # Aquí empezamos a hacer la gráfica 3D
        fig = plt.figure(figsize=(12, 9))
        ax = fig.add_subplot(111, projection='3d')

        # Dibujamos la espira, la carga y los vectores del campo y fuerza
        ax.plot(espira[:, 0], espira[:, 1], espira[:, 2], 'b', label='Espira con corriente')
        ax.scatter(*r_carga, color='red', s=60, label='Carga puntual q')
        ax.quiver(*r_carga, *B_carga, color='green', length=0.3, normalize=True, label='B en carga')
        ax.quiver(*r_carga, *F_magn, color='magenta', length=0.3, normalize=True, label='F = q v × B')
        ax.quiver(X, Y, Z, BX, BY, BZ, color='orange', length=0.2, normalize=True, alpha=0.6)

        # Ponemos etiquetas y título
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.set_zlabel('Z [m]')
        ax.set_title('Campo magnético generado por una espira + Fuerza sobre carga')
        ax.legend()
        ax.set_box_aspect([1,1,1])  # Para que el gráfico no se deforme
        ax.grid(True)

        plt.show()  # Mostramos la gráfica
        print("✅ ¡Gráfica generada!")

# Esta función detecta cuando eliges “Sí” en el botón para confirmar la simulación
def confirmar(change):
    if change['new'] == "Sí":
        simular()  # Llama a la función que hace la simulación y gráfica

# Conectamos los botones con las funciones para que respondan a clics o cambios
ready_button.on_click(on_ready_clicked)
confirm_button.observe(confirmar, names='value')

# Finalmente mostramos toda la interfaz para que puedas interactuar
display(
    widgets.HTML("<h3>🔧 Introduce los parámetros del sistema</h3>"),
    R_input, I_input, q_input,
    x_input, y_input, z_input,
    vx_input, vy_input, vz_input,
    ready_button,
    output
)


HTML(value='<h3>🔧 Introduce los parámetros del sistema</h3>')

FloatText(value=1.0, description='Radio R (m):')

FloatText(value=1.0, description='Corriente I (A):')

FloatText(value=1e-06, description='Carga q (C):')

FloatText(value=0.0, description='x carga:')

...

FloatText(value=0.0, description='vx:')

FloatText(value=0.0, description='vy:')

FloatText(value=0.0, description='vz:')

Button(button_style='success', description='Listo ✅', style=ButtonStyle())

Output()